In [1]:
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import json

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

In [2]:
#extract credentials from environment variabes and instantiate session
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
#grab playlists for user
myPlaylists = sp.user_playlists("ben.doan4366")

#extract tracks for gym and cooking playlists
gymPlaylist = sp.playlist_tracks("1Vu8EHOlNPBFVBxmKtGHlr")
cookingPlaylist = sp.playlist_tracks("5myx61jwlbF2iMDVWNK0Ps")

In [74]:
#sample raw repsonse from Spotify
gymPlaylist

{'href': 'https://api.spotify.com/v1/playlists/1Vu8EHOlNPBFVBxmKtGHlr/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2016-09-12T21:45:52Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/ben.doan4366'},
    'href': 'https://api.spotify.com/v1/users/ben.doan4366',
    'id': 'ben.doan4366',
    'type': 'user',
    'uri': 'spotify:user:ben.doan4366'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'},
       'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG',
       'id': '6oMuImdp5ZcFhWP0ESe6mG',
       'name': 'Migos',
       'type': 'artist',
       'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
     

In [138]:
#function to transition raw json response from Spotify to dataframe of artist and so

def prepArtistStagingTables(raw_playlist_dictionary):
    
    #trim meta data and extract track field from raw dictionary
    cleaned_playlist_dict = [sub['track'] for sub in raw_playlist_dictionary['items']]
    
    #generate raw dataframe
    playlist_df_raw = pd.read_json(json.dumps(cleaned_playlist_dict))
    
    #extract artist column from pd
    artist_df = playlist_df_raw.loc[:,["name","artists"]].explode("artists")
    
    #extract artist dictionary values into series, and recast to dataframe 
    artist_fields_expanded = artist_df["artists"].apply(pd.Series)
    artist_fields_expanded.rename(columns={"name":"artist_name"}, inplace=True)
    
    #concat recasted dataframe to original dataframe with song name
    artist_df_final = pd.concat([artist_df, artist_fields_expanded], axis=1).loc[:,["name", "id", "artist_name", "type", "uri"]]
    
    return artist_df_final

In [151]:
def prepAlbumStagingTables(raw_playlist_dictionary):
    #trim meta data and extract item field from raw dictionary
    cleaned_playlist_dict = [sub['track'] for sub in raw_playlist_dictionary['items']]
    
    #generate raw dataframe
    playlist_df_raw = pd.read_json(json.dumps(cleaned_playlist_dict))
    
    print(playlist_df_raw)
    
    #extract artist column from pd
    album_df = playlist_df_raw.loc[:,["name", "album"]].explode("album")
    
    #extract artist dictionary values into series, and recast to dataframe 
    album_fields_expanded = album_df["album"].apply(pd.Series)
    album_fields_expanded.rename(columns={"name":"album_name"}, inplace=True)
    
    #concat recasted dataframe to original dataframe with song name
    artist_df_final = pd.concat([album_df, album_fields_expanded], axis=1).loc[:,["name", "id", "album_name", "artists", "uri"]]
    
    return artist_df_final

In [142]:
gym_playlist_df = prepArtistStagingTables(gymPlaylist)
cooking_playlist_df = prepArtistStagingTables(cookingPlaylist)

In [143]:
final_artist_playlist = pd.concat([gym_playlist_df, cooking_playlist_df])
final_artist_playlist

,name,id,artist_name,type,uri
0,Fight Night,6oMuImdp5ZcFhWP0ESe6mG,Migos,artist,spotify:artist:6oMuImdp5ZcFhWP0ESe6mG
1,Pipe It Up,6oMuImdp5ZcFhWP0ESe6mG,Migos,artist,spotify:artist:6oMuImdp5ZcFhWP0ESe6mG
2,pick up the phone,50co4Is1HCEo8bhOyUWKpn,Young Thug,artist,spotify:artist:50co4Is1HCEo8bhOyUWKpn
2,pick up the phone,0Y5tJX1MQlPlqiwlOH1tJY,Travis Scott,artist,spotify:artist:0Y5tJX1MQlPlqiwlOH1tJY
3,"Floyd Mayweather (feat. Travis Scott, Gucci Mane and Gunna)",50co4Is1HCEo8bhOyUWKpn,Young Thug,artist,spotify:artist:50co4Is1HCEo8bhOyUWKpn
...,...,...,...,...,...
95,Think,536BYVgOnRky0xjsPT96zl,Two Door Cinema Club,artist,spotify:artist:536BYVgOnRky0xjsPT96zl
96,All Yours,16oEz0ihXl8elwxOS0YMNF,APRE,artist,spotify:artist:16oEz0ihXl8elwxOS0YMNF
97,DaNcing in a RoOm,6JINIoM2HUR5ZNTJwoR9eC,EZI,artist,spotify:artist:6JINIoM2HUR5ZNTJwoR9eC
98,Enginn eins og þú,21SqAOtuXFpwygK1GNxjTm,Auður,artist,spotify:artist:21SqAOtuXFpwygK1GNxjTm


In [152]:
gym_playlist_2 = prepAlbumStagingTables(gymPlaylist)

KeyError: 0

In [153]:
test = [sub['track'] for sub in gymPlaylist['items']]
    
#generate raw dataframe
playlist_df_raw = pd.read_json(json.dumps(test))
    
playlist_df_raw

,album,artists,available_markets,disc_number,duration_ms,episode,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'}, 'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG', 'id': '6oMuImdp5ZcFhWP0ESe6mG', 'name': 'Migos', 'type': 'artist', 'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BW', 'BY', 'BZ', 'CA', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MG', 'MK', 'ML', 'MN', ...], 'external_urls': {'spotify': 'https://open.spotify.com/album/56PJDByaunMWwCqs5rV3Nc'}, 'href': 'https://api.spotify.com/v1/albums/56PJDByaunMWwCqs5rV3Nc', 'id': '56PJDByaunMWwCqs5rV3Nc', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273c0f275798fc87a26c3d3c5e4', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e02c0f275798fc87a26c3d3c5e4', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d00004851c0f275798fc87a26c3d3c5e4', 'width': 64}], 'name': 'No Label II', 'release_date': '2014-06-03', 'release_date_precision': 'day', 'total_tracks': 20, 'type': 'album', 'uri': 'spotify:album:56PJDByaunMWwCqs5rV3Nc'}","[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'}, 'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG', 'id': '6oMuImdp5ZcFhWP0ESe6mG', 'name': 'Migos', 'type': 'artist', 'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'}]","[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, BB, BD, BE, BF, BG, BH, BI, BJ, BN, BO, BR, BS, BW, BY, BZ, CA, CH, CI, CL, CM, CO, CR, CV, CW, CY, CZ, DE, DJ, DK, DM, DO, DZ, EC, EE, EG, ES, FI, FJ, FM, FR, GA, GB, GD, GH, GM, GN, GQ, GR, GT, GW, HK, HN, HR, HT, HU, ID, IE, IL, IN, IS, IT, JM, JO, JP, KE, KG, KH, KM, KN, KR, KW, KZ, LA, LB, LC, LK, LR, LS, LT, LU, LV, MA, MC, MD, ME, MG, MK, ML, MN, ...]",1,216247,False,True,{'isrc': 'QMCE31400019'},{'spotify': 'https://open.spotify.com/track/2n5gVJ9fzeX2SSWlLQuyS9'},https://api.spotify.com/v1/tracks/2n5gVJ9fzeX2SSWlLQuyS9,2n5gVJ9fzeX2SSWlLQuyS9,False,Fight Night,68,https://p.scdn.co/mp3-preview/2a7b91b3e760629de9e564cf701ec658c633b453?cid=048069deb13641da84c999f457c3963a,True,9,track,spotify:track:2n5gVJ9fzeX2SSWlLQuyS9
1,"{'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'}, 'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG', 'id': '6oMuImdp5ZcFhWP0ESe6mG', 'name': 'Migos', 'type': 'artist', 'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', ...], 'external_urls': {'spotify': 'https://open.spotify.com/album/0sv4nM5FtA8Y3DrvG4CXH8'}, 'href': 'https://api.spotify.com/v1/albums/0sv4nM5FtA8Y3DrvG4CXH8', 'id': '0sv4nM5FtA8Y3DrvG4CXH8', 'images': [

In [179]:
album_df = playlist_df_raw.loc[:,["name", "album"]]
album_df_final = album_df["album"].apply(pd.Series)
album_df_final.rename(columns={"name":"album_name"}, inplace=True)

album_df_final_2 = album_df_final.explode("artists")
album_df_final_3 = album_df_final_2["artists"].apply(pd.Series)
album_df_final_3.rename(columns={"name":"artist_name"}, inplace=True)

album_concat_f = pd.concat([album_df_final_3, album_df_final_2, album_df], axis=1)
album_concat_f

,external_urls,href,id,artist_name,type,uri,album_type,artists,available_markets,external_urls,...,id,images,album_name,release_date,release_date_precision,total_tracks,type,uri,name,album
0,{'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'},https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG,6oMuImdp5ZcFhWP0ESe6mG,Migos,artist,spotify:artist:6oMuImdp5ZcFhWP0ESe6mG,album,"{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'}, 'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG', 'id': '6oMuImdp5ZcFhWP0ESe6mG', 'name': 'Migos', 'type': 'artist', 'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'}","[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, BB, BD, BE, BF, BG, BH, BI, BJ, BN, BO, BR, BS, BW, BY, BZ, CA, CH, CI, CL, CM, CO, CR, CV, CW, CY, CZ, DE, DJ, DK, DM, DO, DZ, EC, EE, EG, ES, FI, FJ, FM, FR, GA, GB, GD, GH, GM, GN, GQ, GR, GT, GW, HK, HN, HR, HT, HU, ID, IE, IL, IN, IS, IT, JM, JO, JP, KE, KG, KH, KM, KN, KR, KW, KZ, LA, LB, LC, LK, LR, LS, LT, LU, LV, MA, MC, MD, ME, MG, MK, ML, MN, ...]",{'spotify': 'https://open.spotify.com/album/56PJDByaunMWwCqs5rV3Nc'},...,56PJDByaunMWwCqs5rV3Nc,"[{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273c0f275798fc87a26c3d3c5e4', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e02c0f275798fc87a26c3d3c5e4', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d00004851c0f275798fc87a26c3d3c5e4', 'width': 64}]",No Label II,2014-06-03,day,20,album,spotify:album:56PJDByaunMWwCqs5rV3Nc,Fight Night,"{'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'}, 'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG', 'id': '6oMuImdp5ZcFhWP0ESe6mG', 'name': 'Migos', 'type': 'artist', 'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BW', 'BY', 'BZ', 'CA', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MG', 'MK', 'ML', 'MN', ...], 'external_urls': {'spotify': 'https://open.spotify.com/album/56PJDByaunMWwCqs5rV3Nc'}, 'href': 'https://api.spotify.com/v1/albums/56PJDByaunMWwCqs5rV3Nc', 'id': '56PJDByaunMWwCqs5rV3Nc', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273c0f275798fc87a26c3d3c5e4', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e02c0f275798fc87a26c3d3c5e4', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d00004851c0f275798fc87a26c3d3c5e4', 'width': 64}], 'name': 'No Label II', 'release_date': '2014-06-03', 'release_date_precision': 'day', 'total_tracks': 20, 'type': 'album', 'uri': 'spotify:album:56PJDByaunMWwCqs5rV3Nc'}"
1,{'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'},https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG,6oMuImdp5ZcFhWP0ESe6mG,Migos,artist,spotify:artist:6oMuImdp5ZcFhWP0ESe6mG,album,"{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'}, 'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG', 'id': '6oMuImdp5ZcFhWP0ESe6mG', 'name': 'Migos', 'type': 'artist', 'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'}","[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, BB, BD, BE, BF, BG, BH, BI, BJ, BN, BO, BR, BS, BT, BW, BY, BZ, CA, CH, CI, CL, CM, CO, CR, CV, CW, CY, CZ, DE, DJ, DK, DM, DO, DZ, EC, EE, EG, ES, FI, FJ, FM, GA, GB, GD, GE, GH, GM, GN, GQ, GR, GT, GW, GY, HK, HN, HR, HT, HU, ID, IE, IL, IN, IS, IT, JM, JO, J

In [171]:
album_concat

,album_type,artists,available_markets,external_urls,href,id,images,name,release_date,release_date_precision,total_tracks,type,uri,name,album
0,album,"[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'}, 'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG', 'id': '6oMuImdp5ZcFhWP0ESe6mG', 'name': 'Migos', 'type': 'artist', 'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'}]","[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, BB, BD, BE, BF, BG, BH, BI, BJ, BN, BO, BR, BS, BW, BY, BZ, CA, CH, CI, CL, CM, CO, CR, CV, CW, CY, CZ, DE, DJ, DK, DM, DO, DZ, EC, EE, EG, ES, FI, FJ, FM, FR, GA, GB, GD, GH, GM, GN, GQ, GR, GT, GW, HK, HN, HR, HT, HU, ID, IE, IL, IN, IS, IT, JM, JO, JP, KE, KG, KH, KM, KN, KR, KW, KZ, LA, LB, LC, LK, LR, LS, LT, LU, LV, MA, MC, MD, ME, MG, MK, ML, MN, ...]",{'spotify': 'https://open.spotify.com/album/56PJDByaunMWwCqs5rV3Nc'},https://api.spotify.com/v1/albums/56PJDByaunMWwCqs5rV3Nc,56PJDByaunMWwCqs5rV3Nc,"[{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273c0f275798fc87a26c3d3c5e4', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e02c0f275798fc87a26c3d3c5e4', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d00004851c0f275798fc87a26c3d3c5e4', 'width': 64}]",No Label II,2014-06-03,day,20,album,spotify:album:56PJDByaunMWwCqs5rV3Nc,Fight Night,"{'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'}, 'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG', 'id': '6oMuImdp5ZcFhWP0ESe6mG', 'name': 'Migos', 'type': 'artist', 'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BW', 'BY', 'BZ', 'CA', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MG', 'MK', 'ML', 'MN', ...], 'external_urls': {'spotify': 'https://open.spotify.com/album/56PJDByaunMWwCqs5rV3Nc'}, 'href': 'https://api.spotify.com/v1/albums/56PJDByaunMWwCqs5rV3Nc', 'id': '56PJDByaunMWwCqs5rV3Nc', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273c0f275798fc87a26c3d3c5e4', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e02c0f275798fc87a26c3d3c5e4', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d00004851c0f275798fc87a26c3d3c5e4', 'width': 64}], 'name': 'No Label II', 'release_date': '2014-06-03', 'release_date_precision': 'day', 'total_tracks': 20, 'type': 'album', 'uri': 'spotify:album:56PJDByaunMWwCqs5rV3Nc'}"
1,album,"[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'}, 'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG', 'id': '6oMuImdp5ZcFhWP0ESe6mG', 'name': 'Migos', 'type': 'artist', 'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'}]","[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, BB, BD, BE, BF, BG, BH, BI, BJ, BN, BO, BR, BS, BT, BW, BY, BZ, CA, CH, CI, CL, CM, CO, CR, CV, CW, CY, CZ, DE, DJ, DK, DM, DO, DZ, EC, EE, EG, ES, FI, FJ, FM, GA, GB, GD, GE, GH, GM, GN, GQ, GR, GT, GW, GY, HK, HN, HR, HT, HU, ID, IE, IL, IN, IS, IT, JM, JO, JP, KE, KG, KH, KI, KM, KN, KR, KW, KZ, LA, LB, LC, LI, LK, LR, LS, LT, LU, LV, MA, MC, MD, ME, ...]",{'spotify': 'https://open.spotify.com/album/0sv4nM5FtA8Y3DrvG4CXH8'},https://api.spotify.com/v1/albums/0sv4nM5FtA8Y3DrvG4CXH8,0sv4nM5FtA8Y3DrvG4CXH8,"[{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273337940daef2b57170edd5c3b', 'width': 640}, {'height': 300, 'url': 'https://i.scd